# Análise da influência do STAT3 em proteínas da via do NF-$\kappa$B
No trabalho laboratorial, o factor de transcrição NF-$\kappa$B será detetado em células HeLa com e sem STAT3, estimuladas com TNF-$\alpha$. O STAT3 é um fator de transcrição que, à semelhança do NF-$\kappa$B, pode ser ativado pelo TNF-$\alpha$ e cuja função inclui o controlo da proliferação, diferenciação, sobrevivência e resposta inflamatória. Sabe-se que estes dois fatores interagem fisicamente na célula e regulam em conjunto vários genes, alguns dos quais participam ativamente nas suas próprias vias. Gostariamos então de responder às seguintes questões:

- Como é que o knock-out (KO) do STAT3 afeta a dinâmica da via do NF-$\kappa$B?
- Será que a retroação negativa é influenciada pela presença de STAT3?
- Será possível usar o modelo Lipniacki et al para prever possíveis perturbações causas pelo KO do STAT3 na via do NF-$\kappa$B?

Problema: O STAT3 não está presente no modelo!

![stat3_modelo_question](figures/stat3_modelo_question.svg "Como ligar o STAT3 ao modelo?")

O modelo de Lipniacki et al é uma representação abstrata e simplificada do sistema vivo, no qual não estão presentes muitos dos componentes que participam na resposta da via do NF-$\kappa$B. O objetivo desta análise é identificar os caminhos mais curtos numa rede de interações que liguem o STAT3 às proteínas representadas no modelo matemático. Para isso vamos:

1. Procurar genes diretamente regulados pelo STAT3 (módulo transcricional):

    1. Identificar componentes da via do NF-$\kappa$B diretamente reguladas pelo STAT3,
    2. Identificar reguladores da via do NF-$\kappa$B diretamente regulados pelo STAT3. <p>

2. Ligar os reguladores aos componentes da via (módulo de sinalização),

3. Juntar os dois módulos (rede completa),

4. Ligar os componentes da via ao modelo matemático.

No final queremos ficar com uma rede de interações com a seguinte estrutura:

![stat3_targets](figures/stat3_targets.svg "Diagrama da rede que queremos obter.")

# Breve recapitulação de conceitos (em atualização...)

## *Pandas*: módulo para manipulação e análise de dados tabulares
Este pequeno resumo não exaustivo pretende relembrar alguns dos conceitos base do módulo *Pandas* e não despensa a consulta da [documentação oficial](https://pandas.pydata.org/pandas-docs/stable/index.html).

### Tipos de objetos no *Pandas*
Em Python, um objeto é uma instância de uma estrutura ou tipo de dados (Classe). Todos os objetos possuem atributos (descritores do seu estado) e métodos (modificadores do seu estado) típicos da sua Classe. Para quem queira aprender mais sobre programação orientada a objetos em Python, este [tutorial](https://realpython.com/python-classes/) é um bom começo.

No `Pandas` existem dois tipos principais de estruturas:

- `Series` -> Vetor de dados indexado unidimensional. Pode ser visto como uma coluna retirada de um `DataFrame` ([Docs](https://pandas.pydata.org/docs/user_guide/dsintro.html#series)).
- `DataFrame` -> Estrutura de dados bidimensional indexada constituida por dois eixos: linhas e colunas. É semelhante a uma folha de cálculo ou tabela e pode ser usado para a mesma finalidade ([Docs](https://pandas.pydata.org/docs/user_guide/dsintro.html#dataframe)).

### Criação e Manipulação de DataFrames
Um objeto da classe `DataFrame` pode ser criado passando à função `DataFrame()` uma outra estrutura de dados (ex: um dicionário). Em alternativa, podemos importar uma tabela de um ficheiro *.txt* ou *.csv* usando as funções `read_table()` ou `read_csv()`.

Uma `Dataframe` possui vários atributos que podem ser facilmente acedidos. Alguns dos mais usados são `DataFrame.shape` que retorna um tuplo (n_row, n_col) com o número de linhas e colunas, e `DataFrame.index` ou `DataFrame.columns`, que devolvem um array com os nomes das linhas e colunas, respetivamente.

Os método são funções próprias de cada objeto usadas para aceder ou modificar o seu estado interno. Por exemplo, `DataFrame.head(n)` permite aceder às primeiras **n** linhas de uma `DataFrame`. A maioria dos métodos aplicados a uma `DataFrame` devolvem uma cópia profunda (ver [shallow copy vs deep copy](https://realpython.com/copying-python-objects/)) e não modificam o objeto original. Os métodos podem ser usados em cadeia, o que permite efetuar várias operações rápidas para selecionar dados. Uma série comum de operações consiste em selecionar um subconjunto de linhas e/ou colunas por indexação, remover linhas duplicadas, criar um novo índice e renomear colunas:
```python
DataFrame.loc[indexing].drop_duplicates().reset_index(drop=True).rename(columns={"old_name":"new_name"})
```

### Indexação
O módulo Pandas é muito flexível e permite facilmente aceder aos elementos das estruturas de dados utilizando vários métodos.

- **Seleção por posição**: `DataFrame.iloc[i, j]` permite aceder ao elemento da linha i e coluna j,
  
- **Seleção por etiquetas (nomes)**: `DataFrame.loc[rowname, colname]`. Para aceder a linhas ou colunas inteiras: `DataFrame.loc[rownames]` e `DataFrame[colnames]`,
  
- **Slices**: podem ser feitos usando posição ou etiquetas. Exemplos de *slices* posicionais: `DataFrame.loc[startrowname : endrowname]`, `DataFrame[startcolname : endcolname]`,
  
- **Indexação Booleana (condicional)**: um dos métodos mais usados para "questionar" uma `DataFrame`. Consiste em indexar com um vetor booleano (do tipo `[True, False,...]`) do mesmo tamanho que o número de linhas. Só as linhas verdadeiras serão selecionadas. Por exemplo, para a `DataFrame` *df*:
    - `df[df["A"] > 1]` devolve as linhas de `df` nas quais a coluna `A` é maior que 1,
    - `df.loc[df["A"] == 1, B]` devolve os elementos da coluna B das linhas em que a coluna `A` é igual a 1,
    - `df[df["A"].isin([1,2])]` devolve as linhas de `df` nas quais a coluna `A` é 1 ou 2.
 
É possível criar questões mais complexas combinando vários vetores booleanos. As linhas selecionadas serão aquelas que sejam verdadeiras em todos os vetores. Para uma explicação mais detalhada consultar [*Indexing and selecting data*](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#).

### Alguns métodos usados neste *notebook*

- `DataFrame.pop(col_name)`: remove e retorna a coluna **col_name**,

- `DataFrame.drop()`: elimina colunas ou linhas,
  
- `DataFrame.assign()`: cria novas colunas,
  
- `Series.astype(type)`: altera o tipo de dados dos objetos de uma `Series`, sempre que possível. Uma operação comum é transformar um vetor booleano num vetor de zeros e uns,

- `Series.str`: atributo que permite manipular strings numa `Series`:
    - `str.contains(pat)`: devolve um vetor booleano que é verdadeiro nos elementos que contém o padrão `pat`,
    - `str.split(pat)`: para cada elemento da `Series` devolve uma lista com as palavras numa string, usando `pat` como separador.<p>

- `Dataframe.explode(column)`: Para uma coluna de listas, transforma cada lista num conjunto de linhas.

    ![explode](figures/explode.svg)
  
- `DataFrame.pipe()`: aplica uma função à `DataFrame`. Permite aplicar funções em cadeia como métodos.
  
### Algumas funções usadas neste *notebook*

- `concat(objs)`: `objs` é uma lista de `Series` ou `DataFrames`. Permite concatenar vários objetos num único e adicionar novas linhas ou colunas. Por exemplo, se tivermos dois `DataFrame` com colunas iguais podemos juntá-los num único:

  ![](figures/concat.svg)

- `merge(left, right)`: *merge* é uma operação de junção de tabelas, típica das bases de dados relacionais com [SQL](https://en.wikipedia.org/wiki/SQL). Permite juntar, pelas colunas, duas `DataFrame` contendo dados diferentes. Mas, ao contrário do *concat*, é necessário que (pelo menos) uma coluna em cada tabela contenha chaves para a junção. A junção pode ser interior (interseção entre chaves), exterior (união entre chaves) ou o produto cartesiano (externo) dos dois vetores de chaves. Numa junção exterior podem ainda ser guardadas apenas as chaves de uma `DataFrame`. Por exemplo:

  ![](figures/merge.svg)

# Módulos a importar

In [1]:
import pandas as pd
import omnipath as op
from gprofiler import GProfiler

# Uma introdução breve à teoria de redes

[Network Science](http://networksciencebook.com/)

(em atualização...)

# 1. Módulo Transcricional

## DoRothEA
[DoRothEA](https://saezlab.github.io/dorothea/articles/dorothea.html) é uma rede de de regulação da transcrição que contém interações fator de transcrição - gene alvo.

![](figures/dorothea1.png)

A cada interação no DoRothEA é atribuído um nível de evidência:
- **A**: Interações curadas manualmente, ou suportadas pelas 4 fontes de evidência, ou por, pelo menos 2 fontes curadas,
- **B-D**: Interações [ChIP-seq](https://en.wikipedia.org/wiki/Chromatin_immunoprecipitation) ou curadas, com informação adicional,
- **E** Interações inferidas por métodos computacionais.

In [2]:
transcriptnet = op.interactions.Transcriptional.get(
    #sources="STAT3", #---------------------> usar para obter apenas os alvos do STAT3
    databases="DoRothEA", #-----------------> Existem várias bases de dados com informação de interação transcricional
    dorothea_levels=["A", "B", "C", "D"], #-> Níveis de evidência que suportam a interações do DoRothEA
    genesymbols=True, #---------------------> Para além dos identificadores uniprot queremos também os gene symbols oficiais 
    organism="human" 
)
print(transcriptnet.shape)
transcriptnet.head()

(278830, 17)


,source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,n_sources,n_primary_sources,n_references,references_stripped
0,P01106,O14746,MYC,TERT,True,True,False,True,True,False,76,CollecTRI:10022128;CollecTRI:10491298;CollecTR...,CollecTRI;DoRothEA;ExTRI_CollecTRI;HTRI_Collec...,15,4,74,10022128;10491298;10606235;10637317;10723141;1...
1,P05412,P84022,JUN,SMAD3,True,False,False,False,False,False,0,NaN,DoRothEA;ReMap_DoRothEA,2,1,None,None
2,P84022,P05412,SMAD3,JUN,True,True,False,True,True,False,3,CollecTRI:10022869;CollecTRI:12374795;DoRothEA...,CollecTRI;DoRothEA;ExTRI_CollecTRI;NTNU.Curate...,10,2,2,10022869;12374795
3,P05412,Q13485,JUN,SMAD4,True,False,False,False,False,False,0,NaN,DoRothEA;RegNetwork_DoRothEA;TRED_DoRothEA,3,1,None,None
4,Q13485,P05412,SMAD4,JUN,True,True,False,True,True,False,3,CollecTRI:10022869;CollecTRI:12374795;DoRothEA...,CollecTRI;DoRothEA;ExTRI_CollecTRI;NTNU.Curate...,10,2,2,10022869;12374795


In [5]:
# Filtrar as interações interactions para incluir apenas os alvos do STAT3
stat3targets = transcriptnet.loc[
    transcriptnet["source_genesymbol"]=="STAT3",
    ["source_genesymbol", "target_genesymbol", "is_stimulation", "is_inhibition"]
]\
.drop_duplicates().reset_index(drop=True).rename(columns={
    "source_genesymbol": "source",
    "target_genesymbol": "target"
})

# Adicionar a coluna effect que descreve o efeito da regulação dos gene pelo STAT3
stat3targets["effect"] =\
    stat3targets.pop("is_stimulation").astype(int) -\
    stat3targets.pop("is_inhibition").astype(int)

# Na ausência de anotação de efeito vamos assumir que o STAT3 promove a transcrição do gene
stat3targets.loc[stat3targets["effect"]==0, "effect"] = 1

# guardar a lista num ficheiro
stat3targets.to_csv("data/stat3_targets.txt", index=False, header=False)
print(len(stat3targets))
stat3targets.head()

643


,source,target,effect
0,STAT3,CDKN1A,1
1,STAT3,PRF1,1
2,STAT3,SOCS3,1
3,STAT3,HSPA4,1
4,STAT3,HP,1


In [4]:
# Adicionalmente vamos guardar uma lista de todos os genes alvo presentes no DoRothEA
dorotheatargets = transcriptnet["target_genesymbol"].drop_duplicates()\
.reset_index(drop=True).rename("dorotheatargets")

# guardar ficheiro
dorotheatargets.to_csv("data/dorothea_targets.txt", index=False, header=False)
print(len(dorotheatargets))
dorotheatargets.head()

18576


0     TERT
1    SMAD3
2      JUN
3    SMAD4
4      FAS
Name: dorotheatargets, dtype: object

## Análise de enriquecimento da Gene Ontology

Vamos recorrer à [Gene Ontology (GO)](https://geneontology.org/) para associar os alvos do STAT3 à via cánonica do NF-$\kappa$B. Uma ontologia é uma representação formal de um domínio do conhecimento, constituída por um grupo de classes ou termos que se relacionam entre si. GO é uma base de conhecimento biológico que possui informações sobre a função de genes em três aspetos ou ontologias distintas:

- **Molecular Function**: Atividades a nível molecular realizadas por produtos de genes. São exemplos de funções moleculares o termo lato *catalytic activity*, assim como termo mais restrito *adenylate cyclase activity*.

- **Cellular Component**: Uma localização relativa a um compartimento ou estrutura, onde se situa uma molécula. São exemplos de componentes celulares os termos *plasma membrane* e *mitochondrial matrix*.

- **Biological Process**: Processos ou programas celulares realizados por várias entidades moleculares. São exemplos de processos biológicos o termo lato *signal transduction* e o termo mais restrito *canonical NF-kappaB signal transduction*.

Uma análise de enriquecimento é realizada para responder à seguinte pergunta: será que o conjunto de entidades que estamos a estudar terá mais elementos de um determinado tipo que o esperado ao acaso? No nosso caso, o conjunto de entidades em estudo são os alvos do STAT3 e os tipo de elementos que queremos ver se estão enriquecidos são os genes anotados em termos GO associados à via do NF-$\kappa$B.

Para responder à pergunta realizamos um teste hipergeométrico. Neste caso, a hipótese nula é que a nossa lista de genes alvo do STAT3 é uma amostra aleatória da população. Para definir a população usamos a distribuição hipergeométrica, que descreve a probabilidade de k sucessos em n acontecimentos sem reposição, numa população de tamanho N contendo K sucessos. No nosso caso:

- k (número de sucessos) - genes da amostra com anotação no termo GO em estudo
- n (tamanho da amostra) - genes alvo do STAT3
- K (sucessos na população) - genes anotados com o termo GO em estudo
- N (tamanho da população) - genes alvo existentes no DoRothEA

![alvos_venn](figures/alvos_venn.svg "Diagrama de Venn que ilustra objetivo da análise de enriquecimento: procura por uma interseção entre os alvos do STAT3 e componentes da via do NF-kB")

No entanto existe um pequeno problema com este método. Estamos a fazer um teste estatístico para cada termo GO ao nível de significância de 0.05. Isso significa que estamos a admitir um risco de 5% de que a nossa descoberta seja um falso positivo. Se realizarmos 100 testes esperamos 5 falsos positivos. Ora se, no total, fizermos 10 descobertas (rejeições da hipótese nula) esperamos que metade dessas descobertas sejam falsos positivos! 

Um dos métodos usados para resolver esta limitação dos testes múltiplos é o controlo da taxa de falsas descobertas (FDR - false discovery rate) para que seja de 5% globalmente.

In [6]:
goea = GProfiler(return_dataframe=True).profile(
    query=stat3targets["target"].tolist(), #--> a nossa lista de genes alvos do STAT3
    organism="hsapiens", #--------------------> organismo de interesse
    sources=["GO:BP"], #----------------------> "Biological Processes (BP)" ramo da Gene Ontology em que nos interessa pesquisar 
    user_threshold= 0.05, #-------------------> nível de significância (p-value < 0.05)
    all_results=True, #-----------------------> inclui os resultados que não são significativos 
    no_evidences=False, #---------------------> inclui as colunas da interseção e das evidências
    domain_scope="annotated", #---------------> usar apenas os genes do nosso universo que estão anotados em GO
    significance_threshold_method="g_SCS", #--> tipo de correção para testes múltiplos
    background=dorotheatargets.tolist() #-----> universo do teste estatístico (todos os alvos no DoRothEA)
)
goea.head()

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,GO:BP,GO:0048522,positive regulation of cellular process,9.008505e-52,True,"""Any process that activates or increases the f...",5523,636,376,18778,0.591195,0.068079,query_1,"[GO:0009987, GO:0048518, GO:0050794]","[CDKN1A, PRF1, SOCS3, HSP90AB1, JUNB, GFAP, VE...","[[IDA, IMP, IEP, IEA], [IDA], [IEA], [IDA, IMP..."
1,GO:BP,GO:0048518,positive regulation of biological process,3.233238e-49,True,"""Any process that activates or increases the f...",6033,636,391,18778,0.614780,0.064810,query_1,"[GO:0008150, GO:0050789]","[CDKN1A, PRF1, SOCS3, HSP90AB1, JUNB, GFAP, VE...","[[IDA, IMP, IEP, IEA], [IDA], [IEA], [IDA, IMP..."
2,GO:BP,GO:0035556,intracellular signal transduction,1.011646e-41,True,"""The process in which a signal is passed on to...",2625,636,230,18778,0.361635,0.087619,query_1,[GO:0007165],"[CDKN1A, SOCS3, VEGFA, SOCS1, ICAM1, AGT, IL10...","[[IDA, IMP, IEP, TAS, IEA], [IEA], [IDA, IMP, ..."
3,GO:BP,GO:0051239,regulation of multicellular organismal process,1.993325e-41,True,"""Any process that modulates the frequency, rat...",2898,636,243,18778,0.382075,0.083851,query_1,"[GO:0032501, GO:0050789]","[CDKN1A, HSP90AB1, JUNB, GFAP, VEGFA, SOCS1, A...","[[IEA], [IDA], [NAS], [IEA], [IDA, IMP, IGI, I..."
4,GO:BP,GO:0023051,regulation of signaling,2.704961e-40,True,"""Any process that modulates the frequency, rat...",3363,636,263,18778,0.413522,0.078204,query_1,"[GO:0023052, GO:0050789]","[SOCS3, HSP90AB1, GFAP, VEGFA, SOCS1, ICAM1, A...","[[IMP, IEA], [IDA], [IEA], [IDA, IMP, IGI, NAS..."


In [7]:
# Vamos procurar os termos GO que contém NF-kappaB no nome e são estatisticamente significativos
# Podemos afirmar que esses termos são constituídos por genes relacionados com a via 
nfkbterms = goea[(goea["name"].str.contains("NF-kappaB"))&(goea["significant"])]
nfkbterms

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
658,GO:BP,GO:0007249,canonical NF-kappaB signal transduction,0.00800,True,"""An intracellular signaling cassette character...",303,636,30,18778,0.047170,0.099010,query_1,[GO:0141124],"[VEGFA, TNF, AKT1, BCL3, ROR1, CTNNB1, IKBKE, ...","[[NAS], [IDA, IMP, ISS, IBA, TAS, NAS, IEA], [..."
723,GO:BP,GO:0043122,regulation of canonical NF-kappaB signal trans...,0.02356,True,"""Any process that modulates the canonical NF-k...",268,636,27,18778,0.042453,0.100746,query_1,"[GO:0007249, GO:1902531]","[VEGFA, TNF, ROR1, CTNNB1, IKBKE, STAT1, ESR1,...","[[NAS], [IDA, IMP, ISS, IBA, TAS, NAS, IEA], [..."


In [8]:
# Os alvos do STAT3 estão enriquecidos em dois termos relacionados com a via canónica
# A coluna intersections contém todos os genes dos termos que nos interessam
term1, term2 = nfkbterms["intersections"]
term1, term2 = set(term1), set(term2)
print(len(term1), len(term2))

30 27


In [9]:
print(term1 >= term2) # testa se o conjunto term2 é um subconjunto de term1
print(term1 - term2) # devolve a diferença entre os dois conjuntos

stat3_nfkb_targets = list(term1 | term2) # calcula a união dos dois conjuntos

# transformar em DataFrame
stat3_nfkb_targets = pd.DataFrame(stat3_nfkb_targets, columns=["target"])

stat3_nfkb_targets

True
{'TLR2', 'AKT1', 'BCL3'}


,target
0,TNIP1
1,TNFRSF10B
2,RORA
3,TLR2
4,HSPB1
5,VEGFA
6,BCL3
7,PRKCE
8,STAT1
9,HMOX1


## A. Identificar proteínas que participam via canónica do NF-$\kappa$B

Não é necessário realizar este passo. O ficheiro com a lista de genes é fornecida com este notebook.

Procurar genes anotados como pertencentes ao termo *Canonical NF-kappaB signal transduction (GO:0007249)*:

* Ir a [AmiGO](https://amigo.geneontology.org/amigo/landing),
* Pesquisar *NF-kappaB* e selecionar `Canonical NF-kappaB signal transduction (GO:0007249)`,
* No menu do lado direito, selecionar as seguintes opções:
    * `aspect` -> "P",
    * `Organism` -> "Homo Sapiens",
    * `Type` -> "protein", <p>

* Clicar o botão `Download` -> em *Selected fields* escolher apenas `Gene/product (bioentity_label)` e clicar outra vez `Download`
* No menu do teu browser escolher  `guardar página como` e escolher o diretório deste notebook.

In [10]:
# ler o ficheiro com os genes do termo Canonical NF-kappaB signal transduction
nfkbgenes = pd.read_table("data/canonical_nfkb_pathway_term.txt", names=["gene"])["gene"]\
.drop_duplicates()
print(len(nfkbgenes))
display(nfkbgenes.head())

# transformar a Series em DataFrame
nfkbgenes = pd.DataFrame(nfkbgenes.rename("target"))
nfkbgenes["protein_type"] = ["pathway"]*len(nfkbgenes)
print(len(nfkbgenes))
nfkbgenes.head()

47


0       TLR2
1       ERC1
2       TLR3
3    NKIRAS1
4     ZNF675
Name: gene, dtype: object

47


,target,protein_type
0,TLR2,pathway
1,ERC1,pathway
2,TLR3,pathway
3,NKIRAS1,pathway
4,ZNF675,pathway


## B. Construção do Módulo Transcricional
A rede que estamos a construir está dividida em dois módulos distintos. Vamos começar por construir o módulo transcricional, que descreve a regulação exercida pelo STAT3 de proteínas da via ou que regulam a via canónica do NF-$\kappa$B.

In [11]:
# lista da interseção da análise de enriquecimento
stat3_nfkb_targets.head()

,target
0,TNIP1
1,TNFRSF10B
2,RORA
3,TLR2
4,HSPB1


In [12]:
transcriptmod = stat3_nfkb_targets.pipe(
    # merge que adiciona a coluna identificadora protein_type e mantém as chaves
    pd.merge, nfkbgenes, on="target", how="left"  
).fillna(
    # preenche os elementos em falta na coluna protein_type
    "regulator", axis=1
).pipe(
    # merge que adiciona a coluna source com a proteína STAT3
    pd.merge, stat3targets, on="target", how="inner")\
 

transcriptmod = transcriptmod[["source", "target", "effect", "protein_type"]]
print(len(transcriptmod))
transcriptmod

30


,source,target,effect,protein_type
0,STAT3,TNIP1,1,regulator
1,STAT3,TNFRSF10B,-1,regulator
2,STAT3,RORA,1,regulator
3,STAT3,TLR2,1,pathway
4,STAT3,HSPB1,1,regulator
5,STAT3,VEGFA,1,regulator
6,STAT3,BCL3,1,pathway
7,STAT3,PRKCE,1,regulator
8,STAT3,STAT1,1,regulator
9,STAT3,HMOX1,1,regulator


# 2. Rede de Sinalização
Uma proteína que regula a via canónica do NF-$\kappa$B pathway não está necessariamente anotada como pertencendo à via. O segundo módulo serve como ligação entre os alvos do STAT3 que são reguladores da via e as proteínas anotadas da via.

## Omnipath
O [OmniPath](https://omnipathdb.org/) é uma base de dados de biologia molecular que integra informação de mais de 100 fontes em 5 recursos:
- rede de sinalização,
- relações enzima - modificação pós-traducional,
- complexos proteicos,
- anotações de proteínas,
- interações intercelulares.

Neste trabalho estaremos apenas interessados na rede de sinalização.

![omnipath](figures/omnipath.svg)

In [13]:
# descarregar a via da sinalização
signalnet = op.interactions.PostTranslational.get(
    directed=True,
    genesymbols=True,
    organism="human"
)
print(len(signalnet))
signalnet.head()

134282


,source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,n_sources,n_primary_sources,n_references,references_stripped
0,P0DP23,P48995,CALM1,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,1,1,3,11290752;11983166;12601176
1,P0DP25,P48995,CALM3,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,1,1,3,11290752;11983166;12601176
2,P0DP24,P48995,CALM2,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,1,1,3,11290752;11983166;12601176
3,Q03135,P48995,CAV1,TRPC1,True,True,False,True,True,False,13,DIP:19897728;HPRD:12732636;IntAct:19897728;Lit...,DIP;HPRD;IntAct;Lit-BM-17;TRIP,5,5,8,10980191;12732636;14551243;16822931;18430726;1...
4,P14416,P48995,DRD2,TRPC1,True,True,False,True,True,False,1,TRIP:18261457,TRIP,1,1,1,18261457


In [15]:
# verificar se existem complexos proteicos na rede
signalnet.loc[
    signalnet["source_genesymbol"].str.contains(r'[:_]')|
    signalnet["target_genesymbol"].str.contains(r'[:_]'),
    ['source_genesymbol', 'target_genesymbol']
].head(5)

,source_genesymbol,target_genesymbol
147,PPP2CA_PPP2R5A_PTPA,RBL1
182,PRKAA1_PRKAA2_PRKAB1_PRKAB2_PRKAG1_PRKAG2_PRKAG3,CRTC2
235,CSNK2A1_CSNK2B,H4C2
247,ATF2_JUN,SELE
252,BTRC_CUL1_SKP1,PER2


In [16]:
# selecionar colunas
signalnetfiltered = signalnet[
["source_genesymbol", "target_genesymbol", "is_stimulation", "is_inhibition"]]\
.rename(columns={"source_genesymbol": "source", "target_genesymbol": "target"})\
.drop_duplicates()

# separar as interações com complexos em interações com proteínas individuais
signalnetfiltered = signalnetfiltered.assign(
    source = signalnetfiltered["source"].str.split("_"),
    target = signalnetfiltered["target"].str.split("_")
)\
.explode("source").explode("target").drop_duplicates()

print(len(signalnetfiltered))
signalnetfiltered.head()

110306


,source,target,is_stimulation,is_inhibition
0,CALM1,TRPC1,False,True
1,CALM3,TRPC1,False,True
2,CALM2,TRPC1,False,True
3,CAV1,TRPC1,True,False
4,DRD2,TRPC1,True,False


In [17]:
# Os genes "source" são as proteínas do módulo transcricional anotadas como reguladoras
source = transcriptmod.loc[transcriptmod["protein_type"]=="regulator", "target"].tolist()

# Os genes "target" são proteínas que pertencem à via do NF-kB
target = nfkbgenes["target"].tolist()

# Filtrar a rede de sinalização com as listas de genes
signalmod = signalnetfiltered[
    (signalnetfiltered["source"].isin(source))&
    (signalnetfiltered["target"].isin(target))
].copy()

# criar coluna de efeito
signalmod["effect"] = signalmod.pop("is_stimulation").astype(int) +\
                        signalmod.pop("is_inhibition").astype(int)
signalmod.loc[signalmod["effect"]==2, "effect"] = 0
print(len(signalmod))
signalmod.head()

60


,source,target,effect
839,TNIP1,IKBKG,1
909,TNFRSF10B,TRADD,1
1106,IKBKE,RELA,0
1783,IKBKE,NFKBIA,0
1870,CTNNB1,EP300,1


# 3. Rede Completa
Neste ponto iremos juntar os dois módulos construídos numa única rede que será exportada para um ficheiro. 
Os ficheiros da rede completa gerados poderão ser utilizados para importar a rede no [Cytoscape](https://cytoscape.org/), que permite a sua visualização e análise.

In [18]:
# o primeiro ficheiro contém as interações da rede
network = pd.concat([
    transcriptmod.drop(columns="protein_type"), signalmod],
    ignore_index=True
)

network["interaction_type"] = ["transcriptional"]*len(transcriptmod) + ["signalling"]*len(signalmod)
network["directed"] = [True]*len(network)
network.to_csv("data/nfkb_network.csv", index=False)
network

,source,target,effect,interaction_type,directed
0,STAT3,TNIP1,1,transcriptional,True
1,STAT3,TNFRSF10B,-1,transcriptional,True
2,STAT3,RORA,1,transcriptional,True
3,STAT3,TLR2,1,transcriptional,True
4,STAT3,HSPB1,1,transcriptional,True
...,...,...,...,...,...
85,CD40,NFKB1,1,signalling,True
86,AKAP13,NFKB1,1,signalling,True
87,IKBKE,NFKB1,1,signalling,True
88,PLCG2,AKT1,0,signalling,True


In [19]:
# também criaremos um ficheiro com os nós (proteínas) da rede e os seus atributos
# começamos por selecionar os nós do módulo transcricional
nodes = pd.concat([
    transcriptmod[["target", "protein_type"]].rename(columns={"target": "protein"}),
    pd.DataFrame({"protein": "STAT3", "protein_type": "TF"}, index=[len(transcriptmod)])
])
print(len(nodes))
nodes.head()

31


,protein,protein_type
0,TNIP1,regulator
1,TNFRSF10B,regulator
2,RORA,regulator
3,TLR2,pathway
4,HSPB1,regulator


In [20]:
# adicionar os nós do módulo de sinalização, que inclui proteínas da via
# e complexos ausentes do módulo transcricional

for proteinlist, proteintype in zip(["source", "target"], ["regulator", "pathway"]):
    # selecionar os genes de uma coluna (source ou target)
    proteins = signalmod[[proteinlist]]\
        .rename(columns={f"{proteinlist}": "protein"})
    # criar coluna
    proteins["protein_type"] = [proteintype]*len(proteins)
    
    # concatenar os novos nós
    nodes = pd.concat([nodes, proteins], ignore_index=True)

nodes.drop_duplicates(inplace=True)
nodes.to_csv("data/network_proteins.csv", index=False)
print(len(nodes))
nodes.tail()

49


,protein,protein_type
118,CHUK,pathway
133,CARD11,pathway
136,MAP3K7,pathway
137,TLR9,pathway
138,RIPK2,pathway


# 4. Correspondência entre o modelo matemático e a via canónica do NF-$\kappa$B
Como referido, o modelo da via do NF-$\kappa$B de Lipniacki et al é uma representação muito simplificada da via nas células. Uma parte significativa das proteínas que pertencem à via estão omitidas no modelo. No entanto, muitos dos componentes da via realizam a sua função em complexos ou participam nas mesmas etapas. Sendo assim, é possível criar uma correspondência direta entre componentes do modelo e componentes da via. Adicionalmente, alguns componentes da via podem ser modelados no modelo através da manipulação de parâmetros.

É muito importante trabalhar com identificadores de genes/proteínas que sejam coerentes e consistentes. Nesta análise utilizámos sempre os identificadores oficiais do [HGNC](https://www.genenames.org/). A correspondência representada na célula abaixo é realizada com estes identificadores.

In [21]:
modelprots = {
    "TNFa": ["TNF"],
    "IKKa": ["CHUK", "IKBKB", "IKBKG"],
    "IkBa": ["NFKBIA"],
    "NFkB": ["RELA", "REL", "NFKB1"],
    "A20": ["TNFAIP3", "TNIP1"]
}